# Direct Stiffness Method

The direct stiffness method is a displacement-based structural analysis method; that is, the primary unkowns are displacement-like quantities i.e. translations and rotations. Force-like quantities are obtained in post-processing. For a linear problem, this method results in a system of linear algebraic equations.

## Example Problem 1

In the cell below, you import the solver to this notebook, and initiate the process of defining all the relevant geometric and material properties of the structure.

In [1]:
# --- Import the solver from /src directiory (Don't touch!) ---
import sys
import os
project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.join(project_path, "src"))
from direct_stiffness_method.direct_stiffness_method import Structure, BoundaryConditions, Solver

import numpy as np

# Elements properties
b = 0.5
h = 1.0
E = 1000
nu = 0.3
A = b*h
Iy = h*b**3/2
Iz = b*h**3/12
J = 0.02861

# Define nodes and their coordinates [#, x, y, z]
nodes = {
    0: [0, 0.0, 0.0, 10.0],
    1: [1, 15.0, 0.0, 10.0],
    2: [2, 15.0, 0.0, 0.0]
}

# Define the connectivity [node i, node j]
elements = [
    [0, 1],
    [1, 2]
]

# Run class
structure = Structure(nodes, elements, E, nu, A, Iy, Iz, J)

In the cell below, you get a summary for the properties you entered.

In [4]:
# Display summary for the geometric and material properties per element
structure.display_summary()

--- Structure Summary ---
Number of Elements: 2
Elasticity Modulus (E): 1000
Poisson's Ratio (nu): 0.3

Element 1:
Element Properties:
Area: 0.5000, Length: 15.0000
Iy: 0.0625, Iz: 0.0417, J: 0.0286
Node 1: (0.0, 0.0, 10.0), Node 2: (15.0, 0.0, 10.0)

Element 2:
Element Properties:
Area: 0.5000, Length: 10.0000
Iy: 0.0625, Iz: 0.0417, J: 0.0286
Node 1: (15.0, 0.0, 10.0), Node 2: (15.0, 0.0, 0.0)

--- Connectivity Matrix ---
Element 1: [0 1]
Element 2: [1 2]

Global Node Numbering:
Global Node 0: Coordinates (0.0, 0.0, 10.0)
Global Node 1: Coordinates (15.0, 0.0, 10.0)
Global Node 2: Coordinates (15.0, 0.0, 0.0)

* * * * * * * * * *


In the below cell, you define the loads and the boundary conditions.

In [7]:
# Define externally applied loads [#, Fx, Fy, Fz, Mx, My, Mz]

loads = {
    0: [0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    1: [1, 0.1, 0.05, -0.07, 0.05, -0.1, 0.25],
    2: [2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
}

# Define supports [#, ux, uy, uz, theta_x, theta_y, theta_z]
# 1 means constrained dof
# 0 means free dof

supports = {
    0: [0, 1, 1, 1, 1, 1, 1],
    1: [1, 0, 0, 0, 0, 0, 0],
    2: [2, 1, 1, 1, 0, 0, 0]
}

bc = BoundaryConditions(loads, supports)

In [9]:
# Compute and print the external load vector
bc.compute_global_load_vector()

# Summarize and print boundary conditions
bc.summarize_boundary_conditions()


--- External Load Vector ---
[[ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.1 ]
 [ 0.05]
 [-0.07]
 [ 0.05]
 [-0.1 ]
 [ 0.25]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]]

--- Boundary Conditions ---
Node 0: Constraints [1, 1, 1, 1, 1, 1]
Node 1: Constraints [0, 0, 0, 0, 0, 0]
Node 2: Constraints [1, 1, 1, 0, 0, 0]


Now, in the below cell, the geometric boundary conditions are applied, then the problem is solved.

In [14]:
solver = Solver(structure, bc)
U_global = solver.solve()
R_global = solver.compute_reactions(U_global)


--- External Load Vector ---
[[ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.1 ]
 [ 0.05]
 [-0.07]
 [ 0.05]
 [-0.1 ]
 [ 0.25]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]]

--- Computed Displacements ---
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 2.86803965e-03]
 [ 1.59168866e+00]
 [-1.28654407e-03]
 [-1.50609630e-01]
 [-3.23213864e-03]
 [ 1.81668866e-01]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.63448485e-01]
 [ 2.04627527e-03]
 [ 1.81668866e-01]]

--- External Load Vector ---
[[ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.1 ]
 [ 0.05]
 [-0.07]
 [ 0.05]
 [-0.1 ]
 [ 0.25]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]]

--- Computed Reactions ---
[[-0.09560132]
 [-0.03395143]
 [ 0.0056728 ]
 [ 0.11048568]
 [-0.02907873]
 [-0.75927148]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.00439868]
 [-0.01604857]
 

## Example Problem 2

In the cell below, you import the solver to this notebook, and initiate the process of defining all the relevant geometric and material properties of the structure.